In [343]:
import random
import torch
from   torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from   torchvision import datasets, transforms

import pandas as pd
import os
import cv2
from os import path as osp
import numpy as np
from torch.utils.data import Dataset, DataLoader

import subprocess
import os
import os.path as osp
from tqdm import tqdm

In [344]:
target_path = '/hdd/datasets/Moments_in_Time_Mini/training'
test_path = '/hdd/datasets/Moments_in_Time_Mini/validation'

'''
for action_folder in os.listdir(test_path):
    action_example_folder = osp.join(test_path, action_folder)

    for action_example in tqdm(os.listdir(action_example_folder)):
        if not action_example.endswith('.mp4'):
            continue
        action_example_name = action_example.split('.')[0]

        write_dir = osp.join(action_example_folder, action_example_name)
        action_example_path = osp.join(action_example_folder, action_example)

        if not osp.exists(write_dir):
            os.makedirs(write_dir)

        run_cmd = 'ffmpeg -loglevel panic -i %s %s' % (action_example_path, write_dir + '/image%d.png')
        os.system(run_cmd)



#open csv
fileIn = open("/hdd/datasets/Moments_in_Time_Mini/validationSet.csv", "r")
#create new csv for writing to 
fileOut = open("/hdd/datasets/Moments_in_Time_Mini/validationSet_filtered.csv", "w")
for row in fileIn:
    #get string from row
    split = row.split(".")
    #delimit directory name from string
    directory = split[0]
    #if it exists in /hdd/datasets/Moments_in_Time_Mini/training, write to filtered csv
    if os.path.isdir("/hdd/datasets/Moments_in_Time_Mini/validation/" + directory):
        #write row to trainingSet_filtered.csv
        fileOut.write(row)
    #else, skip over the row
    
fileIn.close()
fileOut.close()
'''

'\nfor action_folder in os.listdir(test_path):\n    action_example_folder = osp.join(test_path, action_folder)\n\n    for action_example in tqdm(os.listdir(action_example_folder)):\n        if not action_example.endswith(\'.mp4\'):\n            continue\n        action_example_name = action_example.split(\'.\')[0]\n\n        write_dir = osp.join(action_example_folder, action_example_name)\n        action_example_path = osp.join(action_example_folder, action_example)\n\n        if not osp.exists(write_dir):\n            os.makedirs(write_dir)\n\n        run_cmd = \'ffmpeg -loglevel panic -i %s %s\' % (action_example_path, write_dir + \'/image%d.png\')\n        os.system(run_cmd)\n\n\n\n#open csv\nfileIn = open("/hdd/datasets/Moments_in_Time_Mini/validationSet.csv", "r")\n#create new csv for writing to \nfileOut = open("/hdd/datasets/Moments_in_Time_Mini/validationSet_filtered.csv", "w")\nfor row in fileIn:\n    #get string from row\n    split = row.split(".")\n    #delimit directory nam

In [345]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

In [346]:
class MomentsDataset(Dataset):
    def __init__(self, data_path):
        self.data_path = data_path
        self.df = pd.read_csv(self.data_path + 'trainingSet_filtered.csv', header=None)
        self.class_to_index = {}
        actions = [file_name.split('/')[0] for file_name in
                list(self.df[self.df.columns[1]])]

        actions = set(actions)

        for i, action in enumerate(actions):
            self.class_to_index[action] = i


    def get_num_classes(self):
        return len(self.class_to_index)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        #print('---- df loc-----')
        #print(self.df.iloc[idx])
        #print('idx', idx)
        file_loc, label, _, _ = self.df.iloc[idx]
        #print('file loc', file_loc)

        full_path = osp.join(self.data_path, 'training', file_loc)
        dir_path = full_path.split('.')[0]

        im_data = []
        images = os.listdir(dir_path)
        for image in images:
            im = cv2.imread(osp.join(dir_path, image))
            resized_image = cv2.resize(im, (128, 128))
            #gray_im = rgb2gray(resized_image)

            im_data.append(resized_image)

        im_data = np.array(im_data)
        im_data = im_data / 255.0
        im_data = im_data[:90]
        if len(im_data) < 90:
            paste_im_data = np.zeros((90, 128, 128, 3))
            paste_im_data[:len(im_data)] = im_data
            paste_im_data[len(im_data):] = im_data[-1]
            im_data = paste_im_data

        index = self.class_to_index[label]

        ret_info = {
                'images': im_data[0],
                'label': index
                }

        return ret_info

In [347]:
class AutoEncoder(nn.Module):
    
    def __init__(self, code_size):
        super().__init__()
        self.code_size = code_size
        
        # Encoder specification
        self.enc_cnn_1 = nn.Conv2d(3, 6, kernel_size=5) #1, 10
        self.pool = nn.MaxPool2d(2, 2)
        self.enc_cnn_2 = nn.Conv2d(6, 16, kernel_size=5) #10, 20
        self.enc_linear_1 = nn.Linear(16 * 29 * 29, 128) #4 * 4 * 20, 50
        self.enc_linear_2 = nn.Linear(128, 84) #120
        self.enc_linear_3 = nn.Linear(84, self.code_size) #50
        
        # Decoder specification
        self.dec_linear_1 = nn.Linear(self.code_size, 84) #self.code_size, 160
        self.dec_linear_2 = nn.Linear(84, IMAGE_SIZE) #160
        
    def forward(self, images):
        code = self.encode(images)
        out = self.decode(code)
        return out, code
    
    def encode(self, images):
        code = self.enc_cnn_1(images)
        code = F.selu(F.max_pool2d(code, 2))
        
        code = self.enc_cnn_2(code)
        code = F.selu(F.max_pool2d(code, 2))
        
        code = code.view(-1, 16*29*29) #[images.size(0), -1]
        code = F.selu(self.enc_linear_1(code))
        code = F.selu(self.enc_linear_2(code))
        code = self.enc_linear_3(code)
        return code
    
    def decode(self, code):
        out = F.selu(self.dec_linear_1(code))
        #print(out)
        #print(out.shape())
        out = F.sigmoid(self.dec_linear_2(out))
        #print(out.shape())
        out = out.view([code.size(0), 1, IMAGE_WIDTH, IMAGE_HEIGHT]) #1
        return out

In [348]:
IMAGE_SIZE = 16384
IMAGE_WIDTH = IMAGE_HEIGHT = 128 #prev 256 

In [349]:
# Hyperparameters
#code_size = 20 #num_classes
num_epochs = 1 #4
batch_size = 129 #128
lr = 0.002
optimizer_cls = optim.Adam

In [350]:
dataset_path = '/hdd/datasets/Moments_in_Time_Mini/'

#print('data set ')
ds = MomentsDataset(dataset_path)

code_size = ds.get_num_classes()
#print(code_size)

#label_count = ds.get_num_classes()
#batch_size = 4
#print('data loader')
dataloader = DataLoader(ds, batch_size=batch_size, shuffle=True, num_workers=1)

# Load data
#train_data = datasets.MNIST('~/data/mnist/', train=True , transform=transforms.ToTensor())
#test_data  = datasets.MNIST('~/data/mnist/', train=False, transform=transforms.ToTensor())
#train_loader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=batch_size, num_workers=4, drop_last=True)

In [351]:
# Instantiate model
autoencoder = AutoEncoder(code_size)
loss_fn = nn.BCELoss()
optimizer = optimizer_cls(autoencoder.parameters(), lr=lr)

In [352]:
'''
for epoch_i in range(N_EPOCHS):
    for batch in dataloader:

        X = batch['images']
        labels = batch['label']

        #X = X.reshape(batch_size, 3, 128, 128, 90)
        X = X.reshape(batch_size, 3, 128, 128)
        X = X.float()
        predicted_labels = model(X)

        labels = labels.long()

        loss = criterion(predicted_labels, labels)

        loss.backward()
        optimizer.step()

        print('Loss: %.5f' % (loss.data.cpu().numpy()))
'''


criterion = nn.CrossEntropyLoss()
# Training loop
for epoch in range(num_epochs):
    print("Epoch %d" % epoch)
    
    #print('before')
    for batch in dataloader:    # train_loader
        #print(batch)
        #raise ValueError()
        
        '''
        out, code = autoencoder(Variable(batch['images'])) #images
        
        optimizer.zero_grad()
        loss = loss_fn(out, images)
        
        loss.backward()
        optimizer.step()
        '''
        
        X = batch['images']
        labels = batch['label']

        #X = X.reshape(batch_size, 3, 128, 128, 90)
        X = X.reshape(batch_size, 3, 128, 128)
        X = X.float()
        #X = X[0:2097152, ...]
        #predicted_labels = autoencoder(X) #model(X)
        out, code = autoencoder(X) #this returns a tupe of the encoder output and decoder output

        labels = labels.long()
        
        #optimizer.zero_grad()
        #print(out)
        #print(out.size())
        #print(labels)
        
        loss = F.binary_cross_entropy(out, X[0:batch_size//3, ...], size_average=False) #input, target
        #loss = loss_fn(out, labels) #images
        #loss = criterion(predicted_labels, labels)

        loss.backward()
        optimizer.step()

        print('Loss: %.5f' % (loss.data.cpu().numpy()))
        
    #print("Loss = %.3f" % loss.data[0])

Epoch 0
Loss: 1470990.75000
Loss: 1520724.62500
Loss: 1470050.25000
Loss: 1454890.62500
Loss: 1429405.25000
Loss: 1422481.87500
Loss: 1375345.00000
Loss: 1346976.50000
Loss: 1404835.75000
Loss: 1330403.50000
Loss: 1288371.25000
Loss: 1310084.00000
Loss: 1259814.75000
Loss: 1364953.37500
Loss: 1281821.62500
Loss: 1312386.87500
Loss: 1323010.37500
Loss: 1530307.50000
Loss: 1316883.12500
Loss: 1381126.00000
Loss: 1635762.00000
Loss: 1382853.75000
Loss: 1530106.62500
Loss: 1793813.00000
Loss: 2019838.00000
Loss: 1649110.62500
Loss: 1579725.75000
Loss: 1610605.75000
Loss: 1775383.50000
Loss: 1568508.50000
Loss: 1446736.62500
Loss: 1393257.75000
Loss: 1597792.75000
Loss: 3028072.25000
Loss: 2084283.62500
Loss: 1600349.37500
Loss: 1873995.25000
Loss: 1621923.12500
Loss: 1615296.00000
Loss: 1508773.12500
Loss: 1636216.37500
Loss: 1708357.12500
Loss: 1583851.00000
Loss: 1484915.12500
Loss: 1522714.25000
Loss: 1639701.25000
Loss: 1712772.87500
Loss: 1769354.75000
Loss: 1695296.87500
Loss: 155566

KeyboardInterrupt: 

In [ ]:
# Try reconstructing on test data
test_image = random.choice(test_data)
test_image = Variable(test_image.view([1, 1, IMAGE_WIDTH, IMAGE_HEIGHT]))
test_reconst, _ = autoencoder(test_image)

In [ ]:
torchvision.utils.save_image(test_image.data, 'orig.png')
torchvision.utils.save_image(test_reconst.data, 'reconst.png')